RNN LSTM for text classification
=============
<span style="color: lightsteelblue;">Deep Learning</span>

The goal of this notebook is to train recurrent neural network with LSTM cell for the purpose of text classification.

## 1. Data preparation

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

# from __future__ import print_function

import collections
import numpy as np
import matplotlib.pyplot as plt
# import random
# import re
# import string
import tensorflow as tf

# from six.moves import range
# from six.moves.urllib.request import urlretrieve

# custom library
from nlp.preparer import Corpus

In [2]:
corpora_paths = ['./articles/good.articles', './articles/bad.articles'] 
corpora = []
lengths = []

for path in corpora_paths:
  corpus = Corpus(path)
  corpora.append(corpus)
  length = [len(article) for article in corpus.articles]
  lengths.append(length)

  print(f"Corpus label: {corpus.label}, ",
        f"length: {len(corpus.articles)} articles, ",
        f"av length: {round(corpus.average_length())} words, ",
        f"max length: {max(length)} words.")
  print(f"Corpus raw article: {corpus.raw[0][:300]}")
  print(f"Corpus data (words): {corpus.articles[0][:50]}\n")

Corpus label: good,  length: 6666 articles,  av length: 321 words,  max length: 4977 words.
Corpus raw article: 3 февраля в большинстве европейских стран закрылось зимнее трансферное окно — период, когда клубы могут заявлять новых футболистов, купленных у других команд. Ценники меняются чуть ли не ежемесячно. На их формирование влияет множество факторов: результативность, желание клубов выгодно заработать, а 
Corpus data (words): ['num', 'февраля', 'в', 'большинстве', 'европейских', 'стран', 'закрылось', 'зимнее', 'трансферное', 'окно', 'период', ',', 'когда', 'клубы', 'могут', 'заявлять', 'новых', 'футболистов', ',', 'купленных', 'у', 'других', 'команд', '.', 'ценники', 'меняются', 'чуть', 'ли', 'не', 'ежемесячно', '.', 'на', 'их', 'формирование', 'влияет', 'множество', 'факторов', 'результативность', ',', 'желание', 'клубов', 'выгодно', 'заработать', ',', 'а', 'также', 'личные', 'амбиции', 'спортсменов', '.']

Corpus label: bad,  length: 7519 articles,  av length: 84 words,  max leng

### Binning articles by their lengths

In [17]:
# print([i for i, length in enumerate(lengths[0]) if length == maxes[0]])

# binning good articles
# num_bins = 100
bin_size = 5
# print(f"Bin size: {bin_size} words")

# hist, bin_edges = np.histogram(lengths[1], bins=num_bins)

bins = {}
for article in corpora[1].articles:
  length = len(article)
  key = length // bin_size 
  if key in bins.keys():
    bins[key].append(article)
  else:
    bins[key] = [article]

print(f"Number of bins: {len(bins)}")
print([{ len(bins[key]) : key } for key in  sorted(bins.keys())])

for article in bins[6]:
  print(f"{len(article)}")


# # visualization
# plt.figure().set_size_inches(15, 5)

# plt.subplot(211)
# plt.title('Lengths distribution: Good articles')
# plt.bar(range(len(hist)), hist)

# plt.subplots_adjust(hspace=.5)

# plt.subplot(212)
# plt.title('Lengths distribution: Bad articles')
# plt.hist(lengths[1], 100)

# plt.show()

Number of bins: 52
[{4: 6}, {31: 7}, {85: 8}, {225: 9}, {355: 10}, {525: 11}, {603: 12}, {724: 13}, {711: 14}, {619: 15}, {562: 16}, {536: 17}, {460: 18}, {376: 19}, {300: 20}, {260: 21}, {206: 22}, {166: 23}, {131: 24}, {121: 25}, {98: 26}, {80: 27}, {79: 28}, {42: 29}, {41: 30}, {25: 31}, {31: 32}, {15: 33}, {19: 34}, {14: 35}, {6: 36}, {9: 37}, {13: 38}, {8: 39}, {7: 40}, {2: 41}, {3: 42}, {1: 43}, {4: 44}, {2: 45}, {4: 46}, {2: 48}, {3: 49}, {2: 50}, {1: 51}, {1: 52}, {1: 53}, {1: 57}, {2: 58}, {1: 60}, {1: 82}, {1: 118}]
33
33
34
32


In [3]:
vocabulary_size = 25000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

# flatten all words into a single bag
all_words = [word for corpus in corpora for words in corpus.articles for word in words]

data, count, dictionary, reverse_dictionary = build_dataset(all_words)

print(f"Top popular words counts: {count[:10]}")
print(f"Example dictionary item: {dictionary['слон']} => {reverse_dictionary[23732]}")

Top popular words counts: [['UNK', 311786], (',', 191060), ('.', 167323), ('в', 121865), ('number', 70531), ('и', 59437), ('на', 47376), ('с', 26509), ('что', 22411), ('по', 21883)]
Example dictionary item: 23732 => слон


Function to generate a training batch for the LSTM model:

In [ ]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]

    self._last_batch = self._next_batch()


  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, self._text[self._cursor[b]]] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch

  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def words(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  words back into its (most likely) word representation."""
  return [reverse_dictionary[c] for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  for b in batches:
    s = [word for word in words(b)]
  return s

train_batches = BatchGenerator(train_data, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_data, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

LSTM Model:

In [ ]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)

    return output_gate * tf.tanh(state), state

  # Input data.
  train_inputs = tf.placeholder(tf.float32, shape=[None, vocabulary_size])
  train_labels = tf.placeholder(tf.float32)

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      labels=train_labels,
      logits=logits
    ))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)

  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
alph = "abcdefghijklmnoprst"
chars = []

for _ in range(10):
  r_char = random.choice(list(alph))
  chars.append(r_char)

print(chars)
print(chars[:3])
print(chars[3:])